In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import numpy as np
import PIL 
import tensorflow as tf


In [30]:
class EmotionDetector:

  def __init__(self, trained_model_path):
    self.model = tf.keras.models.load_model(trained_model_path)


  #Should be the path to a cropped face, majority of the image should be face if not not likely to work well,
  #(can easily be changed to batch of images if necessary)
  def get_emotion(self, path_to_cropped_face):
    pil_image = PIL.Image.open(path_to_cropped_face).convert('L')

    #reshape the pil image
    pil_image = pil_image.resize((48,48))

    #convert to numpy array
    to_model = np.array(pil_image).reshape((1,48,48,1))

    #return the softmax output, in the following order: 1. Angry, 2. Disgust, 3. Fear, 4. Happy, 5. Neutral, 6. Sad
    #7.Surprise

    #For now, extract the 1d array of emotion probabilities as per the order above, can be changed as necessary if
    #input is batch
    pred = self.model(to_model).numpy()[0]

    return pred
  
  def analyse_emotion(self, path_to_user):
    preds_str = []
    interps = []

    #can edit this as necessary, for example remove disgust and apply softmax as necessary, if delete emotions,
    #adjust the index as necessary, alphabetical order is preserved due to how keras reads in directories
    idx2emotion = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'neutral', 5:'sad', 6:'surprise'}
    overall_emotion = {'angry':0, 'disgust':0, 'fear':0, 'happy':0, 'neutral':0, 'sad':0, 'surprise':0}
    #get all the images in the path
    for item in os.listdir(path_to_user):
      cropped_img_path = os.path.join(path_to_user, item)
      if os.path.isfile(cropped_img_path):
        
        pred = self.get_emotion(cropped_img_path)
        #convert the predictions to indexes
        idx = np.argmax(pred)
        #delete emotions as necessary

        #run softmax function IF DELETE INDEX, ELSE NN output is already normalized
        #pred = np.exp(pred)/np.sum(pred)

        emotion = idx2emotion[idx]
        preds_str.append(emotion)

        #create the dictionary for the interps
        current_iteration = {}

        for i,score in enumerate(list(pred)):
          emotion = idx2emotion[i]
          current_iteration[emotion] = score
          overall_emotion[emotion] += score
        
        interps.append(current_iteration)
      
    
    overall_face_emotion = sorted(overall_emotion.items(), key=lambda x:x[1], reverse = True)[0][0]

    return overall_face_emotion, preds_str, interps 
    



         
      

In [28]:
model = EmotionDetector(r'src/Mood_detector/trained_emotion.keras')

In [12]:
sum(model.get_emotion('uploads/test/img/crops/face/video_1.jpg'))

1.0000000168179213

In [29]:
model.analyse_emotion('uploads/test/img/crops/face')

uploads/test/img/crops/facevideo_1.jpg
uploads/test/img/crops/facevideo_10.jpg
uploads/test/img/crops/facevideo_2.jpg
uploads/test/img/crops/facevideo_3.jpg
uploads/test/img/crops/facevideo_4.jpg
uploads/test/img/crops/facevideo_5.jpg
uploads/test/img/crops/facevideo_6.jpg
uploads/test/img/crops/facevideo_7.jpg
uploads/test/img/crops/facevideo_8.jpg
uploads/test/img/crops/facevideo_9.jpg


('neutral',
 ['neutral',
  'neutral',
  'sad',
  'happy',
  'happy',
  'neutral',
  'fear',
  'neutral',
  'sad',
  'sad'],
 [{'angry': 0.026952071,
   'disgust': 3.3114443e-06,
   'fear': 0.04647292,
   'happy': 0.0010000556,
   'neutral': 0.74368906,
   'sad': 0.18170615,
   'surprise': 0.00017645482},
  {'angry': 0.00598501,
   'disgust': 2.6504136e-08,
   'fear': 0.019079035,
   'happy': 7.7492135e-05,
   'neutral': 0.8727128,
   'sad': 0.102134235,
   'surprise': 1.1328599e-05},
  {'angry': 0.03692203,
   'disgust': 0.00027440983,
   'fear': 0.09771531,
   'happy': 0.0028511651,
   'neutral': 0.1370125,
   'sad': 0.72422576,
   'surprise': 0.0009988273},
  {'angry': 9.94471e-09,
   'disgust': 2.7109953e-17,
   'fear': 8.409566e-08,
   'happy': 0.99999845,
   'neutral': 1.4341771e-06,
   'sad': 3.3089805e-08,
   'surprise': 3.176606e-08},
  {'angry': 0.0022087698,
   'disgust': 7.3140285e-07,
   'fear': 0.005330407,
   'happy': 0.9560441,
   'neutral': 0.020052463,
   'sad': 0.0156